<blockquote>
<h1>Part 1</h1><br />
On the way to your <a href="https://en.wikipedia.org/wiki/Gravity_assist">gravity assist</a> around the Moon, your ship computer beeps angrily about a "<a href="https://www.hq.nasa.gov/alsj/a11/a11.landing.html#1023832">1202 program alarm</a>". On the radio, an Elf is already explaining how to handle the situation: "Don't worry, that's perfectly norma--" The ship computer <a href="https://en.wikipedia.org/wiki/Halt_and_Catch_Fire">bursts into flames</a>.</p>
<p>You notify the Elves that the computer's <a href="https://en.wikipedia.org/wiki/Magic_smoke">magic smoke</a> seems to have <span title="Looks like SOMEONE forgot to change the switch to 'more magic'.">escaped</span>. "That computer ran <em>Intcode</em> programs like the gravity assist program it was working on; surely there are enough spare parts up there to build a new Intcode computer!"</p>
<p>An Intcode program is a list of <a href="https://en.wikipedia.org/wiki/Integer">integers</a> separated by commas (like <code>1,0,0,3,99</code>).  To run one, start by looking at the first integer (called position <code>0</code>). Here, you will find an <em>opcode</em> - either <code>1</code>, <code>2</code>, or <code>99</code>. The opcode indicates what to do; for example, <code>99</code> means that the program is finished and should immediately halt. Encountering an unknown opcode means something went wrong.</p>
<p>Opcode <code>1</code> <em>adds</em> together numbers read from two positions and stores the result in a third position. The three integers <em>immediately after</em> the opcode tell you these three positions - the first two indicate the <em>positions</em> from which you should read the input values, and the third indicates the <em>position</em> at which the output should be stored.</p>
<p>For example, if your Intcode computer encounters <code>1,10,20,30</code>, it should read the values at positions <code>10</code> and <code>20</code>, add those values, and then overwrite the value at position <code>30</code> with their sum.</p>
<p>Opcode <code>2</code> works exactly like opcode <code>1</code>, except it <em>multiplies</em> the two inputs instead of adding them. Again, the three integers after the opcode indicate <em>where</em> the inputs and outputs are, not their values.</p>
<p>Once you're done processing an opcode, <em>move to the next one</em> by stepping forward <code>4</code> positions.</p>
<p>For example, suppose you have the following program:</p>
<pre><code>1,9,10,3,2,3,11,0,99,30,40,50</code></pre>
<p>For the purposes of illustration, here is the same program split into multiple lines:</p>
<pre><code>1,9,10,3,
2,3,11,0,
99,
30,40,50
</code></pre>
<p>The first four integers, <code>1,9,10,3</code>, are at positions <code>0</code>, <code>1</code>, <code>2</code>, and <code>3</code>. Together, they represent the first opcode (<code>1</code>, addition), the positions of the two inputs (<code>9</code> and <code>10</code>), and the position of the output (<code>3</code>).  To handle this opcode, you first need to get the values at the input positions: position <code>9</code> contains <code>30</code>, and position <code>10</code> contains <code>40</code>.  <em>Add</em> these numbers together to get <code>70</code>.  Then, store this value at the output position; here, the output position (<code>3</code>) is <em>at</em> position <code>3</code>, so it overwrites itself.  Afterward, the program looks like this:</p>
<pre><code>1,9,10,<em>70</em>,
2,3,11,0,
99,
30,40,50
</code></pre>
<p>Step forward <code>4</code> positions to reach the next opcode, <code>2</code>. This opcode works just like the previous, but it multiplies instead of adding.  The inputs are at positions <code>3</code> and <code>11</code>; these positions contain <code>70</code> and <code>50</code> respectively. Multiplying these produces <code>3500</code>; this is stored at position <code>0</code>:</p>
<pre><code><em>3500</em>,9,10,70,
2,3,11,0,
99,
30,40,50
</code></pre>
<p>Stepping forward <code>4</code> more positions arrives at opcode <code>99</code>, halting the program.</p>
<p>Here are the initial and final states of a few more small programs:</p>
<ul>
<li><code>1,0,0,0,99</code> becomes <code><em>2</em>,0,0,0,99</code> (<code>1 + 1 = 2</code>).</li>
<li><code>2,3,0,3,99</code> becomes <code>2,3,0,<em>6</em>,99</code> (<code>3 * 2 = 6</code>).</li>
<li><code>2,4,4,5,99,0</code> becomes <code>2,4,4,5,99,<em>9801</em></code> (<code>99 * 99 = 9801</code>).</li>
<li><code>1,1,1,4,99,5,6,0,99</code> becomes <code><em>30</em>,1,1,4,<em>2</em>,5,6,0,99</code>.</li>
</ul>
<p>Once you have a working computer, the first step is to restore the gravity assist program (your puzzle input) to the "1202 program alarm" state it had just before the last computer caught fire. To do this, <em>before running the program</em>, replace position <code>1</code> with the value <code>12</code> and replace position <code>2</code> with the value <code>2</code>. <em>What value is left at position <code>0</code></em> after the program halts?</p>
</blockquote>

In [1]:
using DelimitedFiles
using Base.Iterators

program = dropdims(open(f->readdlm(f, ',', Int64), "Inputs/02.1_Intcode_Program.txt"), dims=1)

instruction = Dict(1 => +, 2 => *)

function restore_1202_program_alarm(program)
    program[2] = 12
    program[3] = 2
    return program
end

restored_gravity_assist_program = restore_1202_program_alarm(program)

function evaluate1(program::Array{Int})
    for (opcode, in_addr1, in_addr2, out_addr) in take(partition(program, 4), length(program) ÷ 4)
        if opcode == 99
            break
        else
            program[out_addr + 1] = instruction[opcode](program[in_addr1 + 1], program[in_addr2 + 1])
        end
    end
    return program
end

@assert evaluate1([1, 9, 10, 3, 2, 3, 11, 0, 99, 30, 40, 50]) == [3500, 9, 10, 70, 2, 3, 11, 0, 99, 30, 40, 50]
@assert evaluate1([1, 0, 0, 0, 99]) == [2, 0, 0, 0, 99]
@assert evaluate1([2, 3, 0, 3, 99]) == [2, 3, 0, 6, 99]
@assert evaluate1([2, 4, 4, 5, 99, 0]) == [2, 4, 4, 5, 99, 9801]
@assert evaluate1([1, 1, 1, 4, 99, 5, 6, 0, 99]) == [30, 1, 1, 4, 2, 5, 6, 0, 99]

println("Restored Gravity Assist Program Output: ", evaluate1(restored_gravity_assist_program)[1])

Restored Gravity Assist Program Output: 3409710


<blockquote>
<h1>Part 2</h1><p>"Good, the new computer seems to be working correctly!  <em>Keep it nearby</em> during this mission - you'll probably use it again. Real Intcode computers support many more features than your new one, but we'll let you know what they are as you need them."</p>
<p>"However, your current priority should be to complete your gravity assist around the Moon. For this mission to succeed, we should settle on some terminology for the parts you've already built."</p>
<p>Intcode programs are given as a list of integers; these values are used as the initial state for the computer's <em>memory</em>. When you run an Intcode program, make sure to start by initializing memory to the program's values. A position in memory is called an <em>address</em> (for example, the first value in memory is at "address 0").</p>
<p>Opcodes (like <code>1</code>, <code>2</code>, or <code>99</code>) mark the beginning of an <em>instruction</em>.  The values used immediately after an opcode, if any, are called the instruction's <em>parameters</em>.  For example, in the instruction <code>1,2,3,4</code>, <code>1</code> is the opcode; <code>2</code>, <code>3</code>, and <code>4</code> are the parameters. The instruction <code>99</code> contains only an opcode and has no parameters.</p>
<p>The address of the current instruction is called the <em>instruction pointer</em>; it starts at <code>0</code>.  After an instruction finishes, the instruction pointer increases by <em>the number of values in the instruction</em>; until you add more instructions to the computer, this is always <code>4</code> (<code>1</code> opcode + <code>3</code> parameters) for the add and multiply instructions. (The halt instruction would increase the instruction pointer by <code>1</code>, but it halts the program instead.)</p>
<p>"With terminology out of the way, we're ready to proceed. To complete the gravity assist, you need to <em>determine what pair of inputs produces the output <code>19690720</code></em>."</p>
<p>The inputs should still be provided to the program by replacing the values at addresses <code>1</code> and <code>2</code>, just like before.  In this program, the value placed in address <code>1</code> is called the <em>noun</em>, and the value placed in address <code>2</code> is called the <em>verb</em>.   Each of the two input values will be between <code>0</code> and <code>99</code>, inclusive.</p>
<p>Once the program has halted, its output is available at address <code>0</code>, also just like before. Each time you try a pair of inputs, make sure you first <em>reset the computer's memory to the values in the program</em> (your puzzle input) - in other words, don't reuse memory from a previous attempt.</p>
<p>Find the input <em>noun</em> and <em>verb</em> that cause the program to produce the output <code>19690720</code>. <em>What is <code>100 * noun + verb</code>?</em> (For example, if <code>noun=12</code> and <code>verb=2</code>, the answer would be <code>1202</code>.)</p>
</blockquote>

## Updated Part 1 Solution (in Light of Part 2 Prompt)

In [2]:
using Base.Iterators
using BenchmarkTools
using DelimitedFiles

gravity_assist_intcode = dropdims(open(f->readdlm(f, ',', Int64), "Inputs/02.1_Intcode_Program.txt"), dims=1)


halt_program = identity
known_instruction = Dict{Int, Tuple{Function, Int}}(1 => (+, 3), 2 => (*, 3), 99 => (halt_program, 0))


struct Instruction
    operator::Function
    n_parameters::Int
    Instruction(operator::Function, n_parameters::Int) = new(operator, n_parameters);
    Instruction(opcode::Int) = Instruction(known_instruction[opcode]...)
end


function setinputs(program_memory::Array{Int, 1}, input_address_values::Int...)
    program_memory = copy(program_memory)
    if length(input_address_values) > 0
        first_instruction::Instruction = Instruction(program_memory[1])
        program_memory[2:2 + first_instruction.n_parameters - 2] = [input_address_values...]
    end
    return program_memory
end


function evaluate(program_memory::Array{Int}, inputs::Int...)
    program_memory = setinputs(program_memory, inputs...)
    instruction_pointer = 1
    while instruction_pointer in 1:length(program_memory)
        instruction = Instruction(program_memory[instruction_pointer])
        if instruction.operator == halt_program
            break
        end
        parameters = program_memory[(instruction_pointer + 1):(instruction_pointer + instruction.n_parameters)] .+ 1
        output_address = pop!(parameters)
        program_memory[output_address] = instruction.operator(program_memory[parameters]...)
        instruction_pointer += instruction.n_parameters + 1
    end
    return program_memory[1]
end

@assert evaluate([1, 9, 10, 3, 2, 3, 11, 0, 99, 30, 40, 50]) == 3500
@assert evaluate([1, 0, 0, 0, 99]) == 2
@assert evaluate([2, 3, 0, 3, 99]) == 2
@assert evaluate([2, 4, 4, 5, 99, 0]) == 2
@assert evaluate([1, 1, 1, 4, 99, 5, 6, 0, 99]) == 30

println("Restored Gravity Assist Program Output: ", evaluate(gravity_assist_intcode, 12, 2))

@benchmark evaluate($(gravity_assist_intcode), 12, 2)

Restored Gravity Assist Program Output: 3409710


BenchmarkTools.Trial: 
  memory estimate:  14.36 KiB
  allocs estimate:  222
  --------------
  minimum time:     61.106 μs (0.00% GC)
  median time:      68.847 μs (0.00% GC)
  mean time:        71.609 μs (0.97% GC)
  maximum time:     2.441 ms (96.57% GC)
  --------------
  samples:          10000
  evals/sample:     1

## Part 2 Solution

In [3]:
function find_noun_and_verb(program_memory)
    for (noun, verb) in collect(product(0:99, 0:99))
        if evaluate(program_memory, noun, verb) == 19_690_720
            return 100 * noun + verb
        end
    end
end

println("100 * noun + verb: ", find_noun_and_verb(gravity_assist_intcode))

100 * noun + verb: 7912
